# Parsing

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import datetime as dt
matplotlib.style.use('ggplot')

In [2]:
car_park={
         "Paseo":0,
         "Marengo":1,
         "Los Robles":2,
         "Holly":3,
         "Del Mar":4}

#column dict
cols=["Park Trans #",
          "Car Park #",
          "Entry Time",
          "Exit Time",
          "Net Price",
          "Net Turnover",
         ]
col_use=cols[1:]

#get rid of parking transaction number

path="D:\Datasets\Del Mar Goldline Station\parsed_mar\\"
all_ent_ppa='all_ENT_PPA.csv'

data=pd.read_csv(path+all_ent_ppa,header=None,names=cols,usecols=col_use)

In [3]:
#Only lookign at delmar for now
data=data[data['Car Park #']==car_park["Del Mar"]]

#convert Nans to 0
data.fillna(0,inplace=True)

#drop price information and car park #
data.drop(['Car Park #','Net Price','Net Turnover'],axis=1,inplace=True)

#to datetime
data['Entry Time']=pd.to_datetime(data['Entry Time'],format='%Y-%m-%d %H:%M:%S')
data['Exit Time']=pd.to_datetime(data['Exit Time'],format='%Y-%m-%d %H:%M:%S')

#remove nonsensical entires with 1994 in Entry or Exit Time
data=data[data['Entry Time']!=pd.to_datetime('1994-01-01 00:00:00')]
data=data[data['Exit Time']!=pd.to_datetime('1994-01-01 00:00:00')]

In [26]:
#Mark as success or failure based on time differential
bin_minutes=10
bin_size=dt.timedelta(0,bin_minutes*60)

# Failure = 1, Success = 0
data['Failed'] = ((data['Exit Time']-data['Entry Time'])<bin_size).astype(int)

In [40]:
num_days=int((max(data['Entry Time'])-min(data['Entry Time'])).days)

In [79]:
data.shape

(536947, 3)

# Plotting

## Total Parking Failure by Month for 2015

In [34]:
matplotlib.rcParams.update({'axes.titlesize': 24})
matplotlib.rcParams.update({'axes.labelsize': 22})
matplotlib.rcParams.update({'xtick.labelsize': 16})
matplotlib.rcParams.update({'ytick.labelsize': 16})
matplotlib.rcParams.update({'legend.fontsize': 16})
matplotlib.rcParams.update({'savefig.dpi': 220})


In [84]:
#remove duplicate 2016
df_short=data.loc[(data['Entry Time'].dt.year==2015) | ((data['Entry Time'].dt.month!=3) & (data['Entry Time'].dt.month!=4))]

#separate out month
by_month=df_short[['Failed']].copy()
temp = pd.DatetimeIndex(df_short['Entry Time'])
by_month['Month'] = temp.month

#add a not failed
by_month['Not Failed'] = (1-by_month['Failed']).astype(int)

#group by month
grouped = by_month.groupby('Month')
bar_data = grouped.aggregate(np.sum)

#month names for axis
months = []
for i in range(1,13):
    months.append(dt.date(2008, i, 1).strftime('%B'))
bar_data.set_index(pd.Series(months),inplace=True)
    
    
bar_data.plot.bar(stacked=True)
plt.title("Number of Cars Who Could Not Find Parking By Month")
#plt.xlabel("Month")
plt.ylabel("Number of Cars")
plt.legend(labels=['Did Not Find Parking','Found Parking'])
plt.gcf().subplots_adjust(bottom=0.15)
plt.show()

## Mean Parking Failure Over the Year

In [65]:
#separate out month
year_data=data[['Failed']].copy()
temp = pd.DatetimeIndex(data['Entry Time'])
year_data['date'] = temp.date

#group by day minutes
year_data=year_data.groupby('date').sum()

year_data.plot(linewidth=3,legend=False)
plt.title("Cars Who Could Not Find Parking Over the Year")
plt.xlabel("")
plt.ylabel("Number of Cars / Day")
plt.show()

In [77]:
year_data[year_data['Failed']>=458]

,Failed
date,
2015-03-05,458
2015-09-24,484
2015-11-18,466
2015-12-10,465
2016-01-25,503
2016-01-26,462
2016-02-01,490
2016-02-05,469
2016-02-23,474


## Mean Parking Failure during the Day

In [86]:
#separate out day
day_data=data[['Failed']].copy()
temp = pd.DatetimeIndex(data['Entry Time'])
day_data['Time'] = temp.time
day_data['Time'] = pd.to_datetime(day_data['Time'], format="%H:%M:%S")

#group by 5 minutes
day_plot=day_data.groupby(pd.TimeGrouper(freq='10Min',key='Time')).mean()

#remove index
day_plot.reset_index(inplace=True)

#remove date
day_plot['Time']=day_plot['Time'].dt.time

#reset index
day_plot.set_index('Time',inplace=True)

day_plot.plot(linewidth=3,legend=False)
plt.title("Cars Who Could Not Find Parking During the Day")
plt.xlabel("Time of Day")
plt.ylabel("Percent of Cars / 10 Minutes")
plt.show()

In [49]:
#group by 10 minutes
day_plot=day_data.groupby(pd.TimeGrouper(freq='10Min',key='Time')).sum()/num_days

#remove index
day_plot.reset_index(inplace=True)

#remove date
day_plot['Time']=day_plot['Time'].dt.time

#reset index
day_plot.set_index('Time',inplace=True)

day_plot.plot(linewidth=3,legend=False)
plt.title("Cars Who Could Not Find Parking During the Day")
plt.xlabel("Time of Day")
plt.ylabel("Number of Cars / 10 Minutes")
plt.show()

## Parking Failure during the Week

In [54]:
#weekday name list
wkdy_name=['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']

#separate out time
week_data=data[['Failed']].copy()
temp = pd.DatetimeIndex(data['Entry Time'])
week_data['date'] = temp.date
week_data['weekday'] = data['Entry Time'].apply(lambda x: wkdy_name[x.weekday()])


week_avg = week_data.groupby(['weekday']).mean()
week_avg=week_avg.reindex(wkdy_name) #get right order

week_avg.plot.bar(stacked=True, legend=False)
plt.title("Cars Who Could Not Find Parking During the Week")
plt.xlabel("")
plt.ylabel("Percent of Cars")
plt.gcf().subplots_adjust(bottom=0.15)
plt.show()

## Histogram of Time Spent In Parking Lot

In [6]:
time_spent = data[['Entry Time','Exit Time']]
time_spent['Total Time']= ((data['Exit Time']-data['Entry Time']).dt.seconds)/60.0 #to convert to minutes

binwidth=30
time_spent.plot.hist(y='Total Time',legend=False,bins=range(int(min(time_spent['Total Time'])), int(max(time_spent['Total Time'])) + binwidth, binwidth))
plt.xticks(np.arange(0, max(time_spent['Total Time']), 60))
plt.title("Time Spent in Del Mark Parking Structure")
plt.xlabel("Time (minutes)")
plt.ylabel("Frequency")
plt.show()

In [ ]:
binwidth=30
time_spent.plot.hist(y='Total Time',legend=False,bins=range(int(min(time_spent['Total Time'])), int(max(time_spent['Total Time'])) + binwidth, binwidth))
plt.xticks(np.arange(0, max(time_spent['Total Time']), 60))
plt.title("Time Spent in Del Mark Parking Structure")
plt.xlabel("Time (minutes)")
plt.ylabel("Frequency")
plt.show()

In [85]:
min(time_spent['Entry Time'])

Timestamp('2015-03-01 00:02:23')

## Combining Capacity and Failure

In [90]:
path="D:\Datasets\Del Mar Goldline Station\parsed_mar\\"

capacity=600

parking_name='del_mar'

car_types=['All Cars','Unknown','Free','Discounted','Full Price']
colnames=['datetime']+car_types

#import
df=pd.read_csv(path+parking_name+'_flow.csv',header=None,names=colnames)
df['datetime']=pd.to_datetime(df['datetime'],format='%Y-%m-%d %H:%M:%S')

#remove march and june from 2016
df=df.loc[(df['datetime'].dt.year==2015) | ((df['datetime'].dt.month!=3) & (df['datetime'].dt.month!=4))]

In [129]:
#data for failure
#separate out day
day_data=data[['Failed']].copy()
temp = pd.DatetimeIndex(data['Entry Time'])
day_data['Time'] = temp.time
day_data['Time'] = pd.to_datetime(day_data['Time'], format="%H:%M:%S")

#group by 10 minutes
day_plot=day_data.groupby(pd.TimeGrouper(freq='10Min',key='Time')).sum()/num_days

#remove index
day_plot.reset_index(inplace=True)

#remove date
day_plot['Time']=day_plot['Time'].dt.time

#reset index
day_plot.set_index('Time',inplace=True)

In [104]:
#separate out time
day_data_cap=df[car_types].copy()
temp = pd.DatetimeIndex(df['datetime'])
day_data_cap['time'] = temp.time

grouped = day_data_cap[['time']+car_types].groupby(['time'])
day_avg_cap = grouped.aggregate(np.mean)/capacity*100


In [139]:
day_plot['Capacity Usage']=day_avg_cap['All Cars']
day_plot.rename(columns={'Failed':'Cannot Find Parking'},inplace=True)

ax=day_plot.plot(secondary_y='Capacity Usage',linewidth=3)
plt.title('Cars Who Cannot Find Parking VS Parking Structure Usage')
ax.set_ylabel('Number of Cars / 10 Minutes')
ax.right_ax.set_ylabel('Parking Structure Usage (%)')
plt.show()

In [137]:
day_plot.rename(columns={'Failed':'Cannot Find Parking'})

,Cannot Find Parking,Capacity Usage
Time,,
00:00:00,1.343434,65.117942
00:10:00,1.262626,65.144809
00:20:00,1.005051,65.180328
00:30:00,0.977273,65.235428
00:40:00,0.934343,65.275501
00:50:00,0.901515,65.325592
01:00:00,0.835859,65.407104
01:10:00,0.797980,65.482696
01:20:00,0.654040,65.541894
